# Aufgabe

Wählt zwei beliebige (englische) Texte aus, von denen ihr erwartet, dass einer etwas "unwahrscheinlicher" für ein Sprachmodell ist als der andere (z.B. ein Gedicht vs. einen Zeitungsartikel). Berechnet die Perplexity des BERT-Modells für beide Texte. Entspricht das Ergebnis eurer Hypothese?

(Gerne könnt ihr die Aufgabe auch für deutsche Texte machen -- dann müsste ihr auf huggingface nach einem deutschen BERT-Modell suchen.)

Siehe dazu auch: https://huggingface.co/docs/transformers/perplexity

In [27]:
#load the texts from the /texts folder
import os
import torch

texts = {}
for file in os.listdir('texts'):
    with open('texts/'+file) as f:
        texts[file] = f.read()
        


In [31]:
# This function was taken from the linked huggingface tutorial https://huggingface.co/docs/transformers/perplexity

def compute_perplexity(model, encodings, device):
    #get the max length of the bert model
    max_length = model.config.max_position_embeddings
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl

In [29]:
#compute the perplexity of the texts using the bert model
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
# Compute perplexity for each text
perplexities = {}
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for name, text in texts.items():
    encodings = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    perplexities[name] = compute_perplexity(model, encodings, device).item()
    print(f'{name} perplexity: {perplexities[name]:.2f}')

battle_hymn_of_the_republic.txt perplexity: 1.45
compcert_c_compiler.txt perplexity: 1.21


c:\Users\theav\miniconda3\envs\sprachverarbeitung\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


# Antwor

Bei den beiden ausgewählten Texten handel es sich um ein Gedicht und einen Hacker News Artikel. Das Gedicht ist "Battle-Hymn of the Republic" von Julia Ward Howe und der Hacker News Artikel ist "The CompCert C compiler" von der compcert.org Webseite.

Meine Erwartung war, dass das Gedicht eine höhere Perplexity hat als der Hacker News Artikel, da das Gedicht in einem älteren Englisch geschrieben ist und dich nicht explizit auf eine perfekte Grammatik konzentriert. Der Hacker News Artikel hingegen ist in einem modernen Englisch geschrieben und sollte daher eine geringere Perplexity haben.

Die Perplexity des Gedichts beträgt 1.45 und die des Hacker News Artikels beträgt 1.21
Damit ist meine Hypothese bestätigt.